In [ ]:
import itertools
from IPython.display import Image
from IPython import display
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import random

from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision

In [ ]:
trans = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                            transforms.Resize((14,28)),
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, ),(0.5, ))])

data = torchvision.datasets.ImageFolder(root = './data',
                                       transform = trans)

test = torchvision.datasets.ImageFolder(root = './val',
                                       transform = trans)

data.classes
test.classes

In [ ]:
batch_size = 50
trn_loader = DataLoader(data,
                        batch_size=batch_size,
                       shuffle = True,
                       drop_last=True)

val_loader = DataLoader(test,
                        shuffle = False)


# dataiter = iter(trn_loader)
# images, labels = dataiter.next()
# print(labels[0])

In [ ]:
# construct model on cuda if available
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

In [ ]:
class CNNClassifier(nn.Module):

    def __init__(self):
        # 항상 torch.nn.Module을 상속받고 시작
        super(CNNClassifier, self).__init__()
        conv1 = nn.Conv2d(1, 6, 5, 1)  # 6@24*24
        # activation ReLU
        pool1 = nn.MaxPool2d(2)  # 6@12*12
        conv2 = nn.Conv2d(6, 16, 5, 1)  # 16@8*8
        # activation ReLU
        pool2 = nn.MaxPool2d(2)  # 16@4*4

        self.conv_module = nn.Sequential(
            conv1,
            nn.ReLU(),
            pool1,
            conv2,
            nn.ReLU(),
            pool2
        )

        fc1 = nn.Linear(64, 32)
        # activation ReLU
        fc2 = nn.Linear(32, 20)
        # activation ReLU
        fc3 = nn.Linear(20, 11)

        self.fc_module = nn.Sequential(
            fc1,
            nn.ReLU(),
            fc2,
            nn.ReLU(),
            fc3
        )

        # gpu로 할당
        if use_cuda:
            self.conv_module = self.conv_module.cuda()
            self.fc_module = self.fc_module.cuda()

    def forward(self, x):
        out = self.conv_module(x)  # @16*4*4
        # make linear
        dim = 1
        for d in out.size()[1:]:  # 16, 4, 4
            dim = dim * d
        out = out.view(-1, dim)
        out = self.fc_module(out)
        return F.softmax(out, dim=1)

cnn = CNNClassifier()

In [ ]:
# loss
criterion = nn.CrossEntropyLoss()
# backpropagation method
learning_rate = 1e-3
optimizer = optim.Adam(cnn.parameters(), lr=learning_rate)
# hyper-parameters
num_epochs = 2
num_batches = len(trn_loader)

trn_loss_list = []
val_loss_list = []
for epoch in range(num_epochs):
    trn_loss = 0.0
    for i, data in enumerate(trn_loader):
        x, label = data
#         if use_cuda:
#             x = x.cuda()
#             label = label.cuda()
        # grad init
        optimizer.zero_grad()
        # forward propagation
        model_output = cnn(x)
        # calculate loss
        loss = criterion(model_output, label)
        # back propagation
        loss.backward()
        # weight update
        optimizer.step()

        # trn_loss summary
        trn_loss += loss.item()
        # del (memory issue)
        del loss
        del model_output

        # 학습과정 출력
#         if (i + 1) % 100 == 0:  # every 100 mini-batches
#             with torch.no_grad():  # very very very very important!!!
#                 val_loss = 0.0
#                 for j, val in enumerate(val_loader):
#                     val_x, val_label = val
#                     if use_cuda:
#                         val_x = val_x.cuda()
#                         val_label = val_label.cuda()
#                     val_output = cnn(val_x)
#                     v_loss = criterion(val_output, val_label)
#                     val_loss += v_loss

#             print("epoch: {}/{} | step: {}/{} | trn loss: {:.4f} | val loss: {:.4f}".format(
#                 epoch + 1, num_epochs, i + 1, num_batches, trn_loss / 100, val_loss / len(val_loader)
#             ))

#             trn_loss_list.append(trn_loss / 100)
#             val_loss_list.append(val_loss / len(val_loader))
#             trn_loss = 0.0

In [ ]:
with torch.no_grad(): # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.
#     X_test, Y_test = val_loader
    for i, data in enumerate(val_loader):
        x_test, Y_test = data

        prediction = cnn(x_test)
        print(prediction)
        correct_prediction = torch.argmax(prediction, 1) == Y_test
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

In [ ]:
# prediction = cnn(images)
# print(prediction)

for data in val_loader:
    x_test, Y_test = data
    
    print('Label: ', Y_test.item())
    single_prediction = cnn(x_test)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())